## Optimal Power Flow
**Power Systems Optimization**

by Michael R. Davidson, Jesse D. Jenkins and Sambuddha Chakrabarti (last updated: October 24, 2022)

This notebook provides an introduction to the Optimal Power Flow (OPF) problem in electric power systems&mdash;which minimizes the short-run production costs of meeting electricity demand at a number of connected locations from a given set of generators subject to various technical and transmission network flow limit constraints. This will be our first treatment of a *network*, which is critical to all power systems.

We will first introduce a model of transmission flows that assumes we can control the flow along each path, in what is called a "**transport model**." This is a straightforward extension to [Economic Dispatch](04-Economic-Dispatch.ipynb) (ED), where we have multiple supply and demand balance constraints at each location or "node" in the network, and a new set of flow constraints between nodes. This is also similar to other common optimization problems such as fleet routing of shipments.

We will then introduce a linear approximation to the optimal power flow problem known as "**DC-OPF**", where we begin to incorporate some of the physics involved in how electricity flows along transmission lines. With this formulation, we recognize that given "injections" (i.e., generation) and "withdrawals" (i.e., demand) of power at each node in the network, flows along lines are not independently controllable. Instead, electrical power flows across transmission lines in relation to their physical properties, namely power flows across parallel circuits or paths in inverse proportion to the [electrical impedance](https://en.wikipedia.org/wiki/Electrical_impedance) of the lines. This can (very frequently) result in hitting flow constraints before we would if could control power flows across all lines as in the transport problem.

This notebook does not explore the full functionality of DC-OPF, which can include inter-temporal constraints, additional generation constraints (e.g., on voltage), security constraints to ensure stability in the case of contingencies, and network losses.

Full "AC optimal power flow" models are also beyond the scope of this notebook, as the full set of physics associated with the interactions of AC flows introduces non-convexities that make this problem much harder to solve. Due to the non-convex nature of the AC power flow problem, simplified formulations that linearize and approximate these complex constraints are frequently employed in power systems operations, including by electricity system operators, which use a linearized "security-constrained" optimal power flow (which ensures power flows would remain simulatenously feasible across a range of possible contingencies) to clear real-time electricity markets.

We will start off with some simple systems, whose solutions can be worked out manually without resorting to any mathematical optimization model and software. But, eventually we will be solving larger system, thereby emphasizing the importance of such software and mathematical models.

## Introduction to OPF


The Optimal Power Flow (OPF) problem is a power system optimal scheduling problem which captures the physics of electricity flows across electricity networks, adding a layer of complexity and more realism to the Economic Dispatch (ED) problem. OPF usually attempts to capture the entire network topology by representing the transmission line interconnections between different nodes (also known as buses, or locations where generators or demand inject or withdraw power into/from the network) including various electrical parameters, such as resistance, series reactance, shunt admittance, etc. The full alternating current or "AC" OPF is a non-convex problem and turns out to be an extremely hard problem to solve (usually NP-hard). Hence, system operators and power marketers usually go about solving a linearized version of it, called the "DC-OPF." The DC-OPF approximation works satisfactorily for bulk power transmission networks as long as such networks are not operated at the brink of instability or under heavily loaded conditions.

## "Transport" model
We will first examine the case where we allow for transmission but ignore the physics of electricity flows, and instead treat it like transporting an ordinary commodity.

$$
\begin{align}
\min \ & \sum_{g \in G} VarCost_g \times GEN_g & \\
\text{s.t.} & \\
 & \sum_{g \in G_i} GEN_g - Demand_i = \sum_{j \in J_i} FLOW_{ij} & \forall \quad i \in \mathcal{N}\\
 & FLOW_{ij} \leq MaxFlow_{ij} & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
 & FLOW_{ij} = - FLOW_{ji} & \forall \quad i, j \in \mathcal{N} \\
 & GEN_g \leq Pmax_g & \forall \quad g \in G \\
 & GEN_g \geq Pmin_g & \forall \quad g \in G  
\end{align}
$$

We introduce a few new **sets** in the above:
- $\mathcal{N}$, the set of all nodes (or buses) in the network where generation, storage, or demand (load) are located
- $J_i \subset \mathcal{N}$, the subset of nodes that are connected to node $i$
- $G_i \subset G$, the subset of generators located at node $i$
 
The **decision variables** in the above problem are:

- $GEN_{g}$, the generation (in MW) produced by each generator, $g$
- $FLOW_{ij}$, the flow (in MW) along the line from $i$ to $j$

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand_i$, the demand (in MW) at bus $i$
- $MaxFlow_{ij}$, the maximum allowable flow along the line from $i$ to $j$
- $VarCost_g$, the variable cost of generator $g$

Notice how the problem above is equivalent to producing a single type of good at a set of factories and shipping them along capacity-limited corridors (roads, rail lines, etc.) to meet a set of demands in other locations. 

### 1. Load packages

In [1]:
import Pkg; Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots; plotly();
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
┌ Info: For saving to png with the Plotly backend PlotlyBase and PlotlyKaleido need to be installed.
└ @ Plots /Users/jdj2/.julia/packages/Plots/W75kY/src/backends.jl:319


### 2. Load and format data

We will load a modified 3-bus case stored in the [MATPOWER case format](https://matpower.org/docs/ref/matpower5.0/caseformat.html). It consists of:

- two generator buses where 1000 MW generators are located, one with variable cost of 50/MWh and another with variable cost of 100/MWh
- one load bus where 600 MW of demand is located
- three lines connecting the buses, each with a maximum flow of 500 MW

The location and numbering of the components:

<img src="img/opf_network.png" style="width: 450px; height: auto" align="left">

In [2]:
datadir = joinpath("OPF_data") 
gen = CSV.read(joinpath(datadir,"gen.csv"), DataFrame);
gencost = CSV.read(joinpath(datadir,"gencost.csv"), DataFrame);
branch = CSV.read(joinpath(datadir,"branch.csv"), DataFrame);
bus = CSV.read(joinpath(datadir,"bus.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gen, gencost, branch, bus]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gen.id = 1:nrow(gen);
gencost.id = 1:nrow(gencost);

# create line ids 
branch.id = 1:nrow(branch);
# add set of rows for reverse direction with same parameters
branch2 = copy(branch)
branch2.f = branch2.fbus
branch2.fbus = branch.tbus
branch2.tbus = branch2.f
branch2 = branch2[:,names(branch)]
append!(branch,branch2)

# Calculate the susceptance of each line, on the assumption that
# reactance is >> resistance, such that we can approximate 
# resistance as = 0 and treat susceptance as the simple  
# reciprocal of reactance (x). 
# See https://en.wikipedia.org/wiki/Susceptance#Relationship_to_reactance
branch.sus = 1 ./ branch.x 

# Here are the buses:
bus

,bus_i,type,pd,qd,gs,bs,area,vm,va,basekv,zone,vmax,vmin
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64
1,1,2,0,0.0,0,0,1,1,0,230,1,1.1,0.9
2,2,2,0,0.0,0,0,1,1,0,230,1,1.1,0.9
3,3,1,600,98.61,0,0,1,1,0,230,1,1.1,0.9


Columns pd and qd indicate the [active and reactive power](https://en.wikipedia.org/wiki/AC_power#Active,_reactive,_and_apparent_power) withdrawal at the bus. (We will ignore qd for this notebook, since we are not considering full AC power flows.) We do not need any of other columns for our purposes.

In [3]:
# This is what the generator dataset looks like:
gen

,bus,pg,qg,qmax,qmin,vg,mbase,status,pmax,pmin,pc1,pc2,qc1min,qc1max,qc2min,qc2max
,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,40,0,30.0,-30.0,1,100,1,1000,0,0,0,0,0,0,0
2,2,170,0,127.5,-127.5,1,100,1,1000,0,0,0,0,0,0,0


In [4]:
# and generator cost dataset:
gencost

,model,startup,shutdown,n,x1,y1,id
,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2,0,0,2,50,0,1
2,2,0,0,2,100,0,2


In the above, model=2 indicates a polynomial variable cost formulation and the column n=2 indicates that there are two terms. Thus, we have a linear cost (in the x1 column) without any quadratic terms (and a zero constant term):

$$
VarCost_g = x1_g
$$

In [5]:
# Here are the transmission lines:
branch

,fbus,tbus,r,x,b,ratea,rateb,ratec,ratio,angle,status,angmin,angmax,id,sus
,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,1,3,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,1,35.5872
2,1,2,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,2,35.5872
3,2,3,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,3,35.5872
4,3,1,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,1,35.5872
5,2,1,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,2,35.5872
6,3,2,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,3,35.5872


Note that while there are three lines, there are six entries here. There is an entry for each 'direction' of flow the lines can accomodate, hence six entries for three lines. The column `fbus` denotes the ID of the "from bus" or origin bus and the column `tbus` denotes the ID of the "to bus" or destination bus (e.g. `fbus`=1, `tbus`=3 is the flow in the direction from bus 1 to bus 3). 

For this transport model formulation, we are only using transmission line capacity (known as "ratings"), given in `ratea`, `rateb`, and `ratec`. These correspond to different ratings based on how long the line might be overloaded, with `ratec` known as an "emergency rating", which could exceed the long-term rating, `ratea`. We will use `ratea` for this model. The dataset also contains resistance and reactance.

### 3. Create solver function (transport)

In [6]:
#=
Function to solve transport flow problem 
Inputs:
    gen -- dataframe with generator info
    branch -- dataframe with transmission lines info
    gencost -- dataframe with generator info
    bus -- dataframe with bus types and loads
Note: it is always a good idea to include a comment blog describing your
function's inputs clearly!
=#
function transport(gen, branch, gencost, bus)
    Transport = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets
      # Set of all generators
    G = gen.id
      # Set of all nodes
    N = bus.bus_i
      # Note: sets J_i and G_i will be described using dataframe indexing below

    # Decision variables   
    @variables(Transport, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        FLOW[N,N]        # flow
        # Note: flow is not constrained to be positive
        # By convention, positive values will indicate flow from the first to second node
        # in the tuple, and a negative flow will indicate flow from the second to the first
        # This matrix is thus "anti-symmetric", which we will ensure with an appropriate
        # constraint.
    end)
                
    # Objective function: minimize sum of generation variable costs for all generators
    @objective(Transport, Min, 
        sum( gencost[g,:x1] * GEN[g] 
                        for g in G)
    )

    # Supply/demand balance constraints, accounting for power flows in/out of each node
    @constraint(Transport, cBalance[i in N], 
        sum(GEN[g] for g in gen[gen.bus .== i,:id]) 
                - bus[bus.bus_i .== i,:pd][1] ==
        sum(FLOW[i,j] for j in branch[branch.tbus .== i,:fbus]))

    # Max generation constraints
    @constraint(Transport, cMaxGen[g in G],
                    GEN[g] <= gen[g,:pmax])

    # Flow constraints on each branch
    for l in 1:nrow(branch)
        @constraint(Transport, 
            FLOW[branch[l,:fbus][1],branch[l,:tbus][1]] <= 
                        branch[l,:ratea])
    end
    
    # Anti-symmetric flow constraints
    @constraint(Transport, cFlowSymmetric[i in N, j in N],
                    FLOW[i,j] == -FLOW[j,i])

    # Solve statement (! indicates runs in place)
    optimize!(Transport)

    # Dataframe of optimal decision variables
    generation = DataFrame(
        id = gen.id,
        node = gen.bus,
        gen = value.(GEN).data
        )
    
    flows = value.(FLOW).data

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        flows,
        cost = objective_value(Transport),
        status = termination_status(Transport)
    )
end

transport (generic function with 1 method)

### 4. Solve

In [7]:
solution = transport(gen, branch, gencost, bus)
solution.generation

Presolving model
2 rows, 4 cols, 6 nonzeros
1 rows, 3 cols, 2 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-20); columns 0(-11); elements 0(-31) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.0000000000e+04
HiGHS run time      :          0.00


,id,node,gen
,Int64,Int64,Float64
1,1,1,600.0
2,2,2,0.0


We generate all 600 MW from Gen A at Bus 1.

In [8]:
solution.flows

3×3 Matrix{Float64}:
   -0.0   100.0  500.0
 -100.0    -0.0  100.0
 -500.0  -100.0   -0.0

In turn, the following flows are created: 

- $l_{13}$ = 500 MW
- $l_{12}$ = 100 MW
- $l_{23}$ = 100 MW

Hence, we are able to maximize the capacity of the line from 1 to 3 ($l_{13}$), and then route the remaining power through Bus 2.

## The "DC" Optimal Power Flow problem

The above model is not physically correct as we cannot arbitrarily route power through lines. We will now introduce a linear approximation to the optimal power flow problem that incorporates this limitation and is tractable and reasonably accurate. This is commonly called the "DC" optimal power flow problem, but in reality (as we'll see below), it is a linearization that still relates to the physics of AC power flows, it just simplifies (and ignores) certain non-convexities to produce a tractable linear programming problem that remains a valid approximation under certain circumstances.

In the "DC" or linear approximation of the AC optimal power flow problem, power flows along a line from bus $i$ to bus $j$ are driven by voltage [phase angle](https://en.wikipedia.org/wiki/Phasor) differences, denoted by $\theta_i$ and $\theta_j$:

$$
FLOW_{ij} = BaseMVA \times B_{ij} (\theta_i-\theta_j)
$$

Where $FLOW_{ij}$ is the flow across the line from node $i$ to node $j$ (in MW), $BaseMVA$ is the base power for the network (in MVA), $B_{ij}$ is the [susceptance](https://en.wikipedia.org/wiki/Susceptance) for the line connecting buses $i$ and $j$ (in per unit terms) and $(\theta_i-\theta_j)$ is the difference in voltage angles between buses (in radians).

Susceptance is the imaginary part of the [admittance](https://en.wikipedia.org/wiki/Admittance) of a line, where [admittance](https://en.wikipedia.org/wiki/Admittance) is a complex number that describes how easy it is for AC current to flow across a given conductor. Power flows in parallel circuits in an AC network in proportion to their admittance (or inverse proportion to [impedance](https://en.wikipedia.org/wiki/Electrical_impedance), which describes how hard it is a measure of the opposition that a circuit presents to a current when a voltage is applied).

Voltage [phase](https://www.allaboutcircuits.com/textbook/alternating-current/chpt-1/ac-phase/) angles describe the displacement of the AC voltage waveform at each node, relative to a reference or "slack" bus. A difference in voltage angles between buses $i$ and $j$ indicates that the peaks and troughs in the sinusoidal voltage waveform at bus $i$ are shifted in time relative to the voltage waveform at bus $j$, as in the image below.

<img src="img/phase_shift.png" style="width: 450px; height: auto"> (*Image source: [allaboutcircuits.com](https://www.allaboutcircuits.com/textbook/alternating-current/chpt-1/ac-phase/)*)

In AC circuits, power flows from nodes with higher voltage angle to buses with lower voltage angle, just as power flows from higher voltage magnitude to lower voltage magnitude in DC circuits.

What causes the shift in voltage phase angle? An AC current flowing across a conductor encountering either [inductive reactance](https://en.wikipedia.org/wiki/Electrical_impedance#Inductive_reactance) (which relates to the magnetizing current, or the energy required to continually induce or establish magnetic fields around a conductor as AC current polarity flips each cycle) or [capacitive reactance](https://en.wikipedia.org/wiki/Electrical_impedance#Capacitive_reactance) (which relates to the charging current, or the energy required to sustain capacitive charges between two conductors separated by an insulator) will experience a shift in the voltage waveform, relative to the current wave form. Inductive reactance causes the voltage waveform to shift forward relative to the current, while capacitive reactive cases the voltage to shift backwards relative to the current. In overhead transmission lines, the primary source of voltage phase angle shifts is the inductance of the transmission lines themselves (although capacitance is relevant for underground lines).

(For (a bit) more on the physics of power flow on AC transmissio lines, you can review [this tutorial from the PJM Interconnection](https://learn.pjm.com/~/media/training/nerc-certifications/gen-exam-materials/bet/20160104-basics-of-elec-power-flow-on-ac.ashx))

The reason this approximation is known as the "DC OPF" is because we ignore [reactive power](https://en.wikipedia.org/wiki/AC_power#Reactive_power_flows) and focus only on flows of real power as in a DC network. But you can see why "DC" OPF is actually a misnomer: we're still dealing with AC voltages and susceptance terms here, we're just linearizing the problem through some simplifying assumptions. In particular, the three basic assumptions used to derive a linearized or "DC" OPF approximation from the underlying AC OPF problem are as follows:

1. The resistance for each branch is negligible relative to the reactance, and can therefore be approximated as ~0.
2. The voltage magnitude at each bus is constant and equal to the base voltage (e.g. equal to 1 p.u).
3. The voltage angle difference $(\theta_j-\theta_i)$ across any branch from bus $i$ to $j$ is sufficiently small such that $cos(\theta_i-\theta_j) \approx 1$ and $sin(\theta_i-\theta_j) \approx (\theta_i-\theta_j)$. Note that $\theta_i,\theta_j$ are measured in [radians](https://en.wikipedia.org/wiki/Radian).

Fortunately, under normal operating conditions, these conditions hold for electricity transmission networks (although note they are not generally acceptable simplifying assumptions for lower voltage distribution networks). 

We can now modify the above transport model to incorporate these new power flow-related decision variables and constraints:

$$
\begin{align}
\min \ & \sum_{g \in G} VarCost_g \times GEN_g & \\
\text{s.t.} & \\
 & \sum_{g \in G_i} GEN_g - Demand_i = \sum_{j\in J(i)} FLOW_{i,j} & \forall \quad i \in \mathcal{N}\\
 & FLOW_{i,j} \leq MaxFlow_{ij} & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
 & FLOW_{i,j} = BaseMVA \times B_{ij}(\theta_i-\theta_j) & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
 & GEN_g \leq Pmax_g & \forall \quad g \in G \\
 & GEN_g \geq Pmin_g & \forall \quad g \in G \\
 & \theta_{slack} = 0
\end{align}
$$

Note that we no longer require a constraint to enforce anti-symmetric flows.

We have the following **sets**:
- $\mathcal{N}$, the set of all nodes (or buses) in the network
- $J_i \subset \mathcal{N}$, the subset of nodes that are connected to node $i$
- $G_i \subset G$, the subset of generators located at node $i$
 
The **decision variables** in the above problem are:

- $GEN_{g}$, the generation (in MW) produced by each generator, $g$
- $\theta_i$, the voltage phase angle at bus $i$ relative to the slack or reference bus ($\theta_{slack}$)
- $FLOW_{i,j}$, the flow from bus i to bus j (in MW)

Note that unlike the transport flow problem, in the OPF problem we *do not* directly choose the flows across lines, but rather choose the real power injections at generator buses via $GEN_{g}$ and the voltage angles $\theta_i$, which collectively determine the power flows across lines via the collection of constraints above. The $FLOW$ decision variable is thus an "auxialiary" variable, as it is precisely determined by the constraint $FLOW_{i,j} = BaseMVA \times B_{ij}(\theta_i-\theta_j)$ for all pairs ($i,j$) for which transmission lines exist. 

(Note that we create $FLOW$ decisions for some pairs of nodes ($i,j$) that are not connected by lines; these variables are "free" variables, as they are not constrained and do not show up in/effect the objective function, so the solver will generally remove these variables in pre-solve step. We could try a different approach to sets for the $FLOW$ variable, e.g. by setting across lines instead of pairs of nodes, to avoid creating these unecessary free variables, but for convenience, we'll create and ignore them for now).

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand_i$, the demand (in MW) at bus $i$
- $MaxFlow_{ij}$, the maximum allowable flow along the line from $i$ to $j$
- $VarCost_g$, the variable cost of generator $g$
- $B_{ij}$, susceptance for line connecting buses $i$ and $j$
- $\theta_{slack}$, the "slack" bus or reference bus from which relative voltage angles at all other buses are calcluated. Thus $\theta_{slack}=0$.
- $BaseMVA$, the base power in MVA for the network (used to scale from standard units to per unit values or vice versa).

### 3. Create solver function (dcopf)

In [9]:
#=
Function to solve DC OPF problem 
Inputs:
    gen -- dataframe with generator info
    branch -- dataframe with transmission lines info
    gencost -- dataframe with generator info
    bus -- dataframe with bus types and loads
Note: it is always a good idea to include a comment blog describing your
function's inputs clearly!
=#
function dcopf(gen, branch, gencost, bus)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets
      # Set of all generators
    G = gen.id
      # Set of all nodes
    N = bus.bus_i
      # sets J_i and G_i will be described using dataframe indexing below

    
    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = gen.mbase[1] # base MVA is 100 MVA for this system
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0
    # Note: by convention this is a generator bus. Hence, we will select bus 1
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gencost[g,:x1] * GEN[g] 
                        for g in G)
    )
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gen[gen.bus .== i,:id]) 
                - bus[bus.bus_i .== i,:pd][1] ==
        sum(FLOW[i,j] for j in branch[branch.fbus .== i,:tbus])
    )
    
    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gen[g,:pmax])

    # Flow constraints on each branch; 
    # In DCOPF, line flow is a function of voltage angles
         # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineFlows = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(branch)) 
    for l in 1:nrow(branch)
        cLineFlows[l] = @constraint(DCOPF, 
            FLOW[branch[l,:fbus],branch[l,:tbus]] == 
            baseMVA*branch[l,:sus]*(THETA[branch[l,:fbus]] - THETA[branch[l,:tbus]])
        )
    end
    
    # Max line flow constraints
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineLimits = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(branch)) 
    for l in 1:nrow(branch)
        cLineLimits[l] = @constraint(DCOPF,
           FLOW[branch[l,:fbus],branch[l,:tbus]] 
           <= branch[l,:ratea]
        ) 
    end

    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        id = gen.id,
        node = gen.bus,
        gen = value.(GEN).data
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = branch.fbus,
        tbus = branch.tbus,
        flow = baseMVA .* branch.sus .* (angles[branch.fbus] .- angles[branch.tbus])
    )
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = bus.bus_i,
        value = dual.(cBalance).data)
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf (generic function with 1 method)

### 4. Solve

In [10]:
solution = dcopf(gen, branch, gencost, bus)
solution.generation

Presolving model
8 rows, 9 cols, 20 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-17); columns 0(-14); elements 0(-34) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.0000000000e+04
HiGHS run time      :          0.00


,id,node,gen
,Int64,Int64,Float64
1,1,1,600.0
2,2,2,0.0


Hence, we still generate all 600 MW from Gen A at Bus 1.

In [11]:
# These are the voltage phase angles of the buses relative to Bus 1.
solution.angles

3-element Vector{Float64}:
  0.0
 -0.0562
 -0.1124

In [12]:
solution.flows

,fbus,tbus,flow
,Int64,Int64,Float64
1,1,3,400.0
2,1,2,200.0
3,2,3,200.0
4,3,1,-400.0
5,2,1,-200.0
6,3,2,-200.0


Thus, we notice that, in contrast to the transport model, we do not max out the capacity of $l_{13}$. The following flows are created:

- $l_{1,3}$ = 400 MW
- $l_{1,2}$ = 200 MW
- $l_{2,3}$ = 200 MW

The reason we can't make maximum use of line $l_{1,3}$ is because power flows split across parallel circuits in inverse proportion to the impedance of the circuit paths. Here, all three branches have equal susceptance, and thus equal impedance (since we are assuming resistance is ~0). Thus, the path $l_{1,2} \rightarrow l_{2,3}$ has twice the impedance as the path $l_{1,3}$ and thus takes half as much power flow coming from Gen A at Bus 1.

### 5. Solve high demand case

Now, let's increase demand at Bus 3 to 800 MW. Despite spare capacity at Gen A, it turns out we will no longer be able to generate all of our power from Gen A alone.

In [16]:
bus_high = copy(bus)
bus_high[3,:pd] = 800 # set demand at bus 3 to 800 MW

sol_high = dcopf(gen, branch, gencost, bus_high)
sol_high.generation

Presolving model
8 rows, 9 cols, 20 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-17); columns 0(-14); elements 0(-34) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  4.5000000000e+04
HiGHS run time      :          0.00


,id,node,gen
,Int64,Int64,Float64
1,1,1,700.0
2,2,2,100.0


This situation is explained by flow patterns, where the capacity of $l_{13}$ is at its maximum, but in order to meet demand at Bus 3, more power needs to be injected in Bus 2, requiring the more costly generator at Bus 2 to dispatch, despite spare capacity at the generator at Bus 1.

In [14]:
sol_high.flows

,fbus,tbus,flow
,Int64,Int64,Float64
1,1,3,500.0
2,1,2,200.0
3,2,3,300.0
4,3,1,-500.0
5,2,1,-200.0
6,3,2,-300.0


In [15]:
sol_high.angles

3-element Vector{Float64}:
  0.0
 -0.056199999999999986
 -0.14049999999999999

The following flows are created: 

- $l_{1,3}$ = 500 MW
- $l_{1,2}$ = 200 MW
- $l_{2,3}$ = 300 MW

What is going on here?

Generator 1 at Bus 1 produces 700 MW, which must split in proportion to impedance once again, with 2/3 of the power or 466.67 MW flowing along $l_{13}$ and 233.33 MW must flow along the route $l_{1,2} \rightarrow l_{2,3}$ with double the impedance. 

At the same time, the 100 MW injected by generator 2 at Bus 2 *also* splits with 2/3 or 66.67 flowing along $l_{2,3}$ and 1/3 or 33.33 along $l_{2,1} \rightarrow l_{1,3}$. 

The total flows across each segment are thus: 

- $l_{1,3}$ = 466.67 + 33.33 = 500 MW
- $l_{1,2}$ = 233.33 - 33.33 = 200 MW
- $l_{2,3}$ = 233. 33 + 66.67 = 300 MW

### 6. Compare prices

The marginal values of the demand constraints at a given bus represent the change in the objective that results from increasing demand at the bus by one unit. This is the natural definition of a "value" of power at that location, and is the basis for **[locational marginal prices](https://www.iso-ne.com/participate/support/faq/lmp)** (LMPs) found in electricity markets.

We examine first the regular case of demand = 600 MW, then the high demand case = 800 MW.

In [17]:
solution.prices

,node,value
,Int64,Float64
1,1,50.0
2,2,50.0
3,3,50.0


All prices are the same in this case. The interpretation: if we were to add an incremental load at any of the buses, we could meet it from additional production from Gen A which has marginal cost of \$50 / MWh. We are not going to hit any transmission limits.

In [18]:
sol_high.prices

,node,value
,Int64,Float64
1,1,50.0
2,2,100.0
3,3,150.0


Something interesting has happened! 

First, note that the prices are different. Hence, we will not be able to meet incremental load from production by Gen A (except if we add load right at Gen A located at Bus 1). Similarly, load at Bus 2 can be met by increasing production from Gen B with marginal cost = \$100 / MWh.

However, why does Bus 3 have a marginal price of \$150 / MWh?! That's higher than the marginal cost of either of our two generators?

The answer lies in what must happen to meet an incremental load at Bus 3 while respecting transmission constraints. We must increase from Gen B, but in doing so, part of the power from Gen B will go through $l_{2,1} \rightarrow l_{1,3}$ in addition to $l_{2,3}$, since power flows split across parallel paths in proportion to admittance (or inverse proportion to impedance). However, without adjusting Gen A's output, an increase in production from Gen B will cause us to exceed the transmission constraint on line $l_{1,3}$, requiring us to throttle back power from Gen A to keep power flows feasible.

The exact change in generation for an incremental 1 MW load at Bus 3 is thus:
- Gen B $\uparrow$ 2 MW
- Gen A $\downarrow$ 1 MW

Hence:

$$
Price_3 = 2 \times VarCost_B - VarCost_A = \$150 \text{ / MWh}
$$

In a network with thousands of nodes and many parallel paths and loop flows, one can see quite quickly how prices may vary in unexpected ways; hence, the need for detailed mathematical models to compute locational marginal prices.

### 6. The IEEE 14 bus test system

We now explore a complicated system, the 14-bus IEEE test system, illustrated here:

<img src="ieee_test_cases/IEEE14BusTestSystem.png" style="width: 450px; height: auto" align="center">

The system consists of:
- 2 generators (located at nodes 1 and 2)
- 11 loads
- a meshed transmission network including transformers and multiple voltages

Our data files for the test system contain parameters for resistance and reactance of the transmission lines, which are related to complex impedance:

$$
Z = R + iX
$$

where $R$ = resistance is the real part, $X$ = reactance is the imaginary part. Recall from above that impedance is the inverse of admittance; hence, we have the following transformation for susceptance:

$$
B = \text{Im}\left(\frac{1}{R + iX}\right) = \frac{-X}{|R + iX|^2} = \frac{-X}{R^2 + X^2}
$$

But, since we neglect the resistance for the purpose of solving the DC-OPF, we can approximate the susceptance from above as:

$$
B = \frac{1}{X}
$$

The data are converted to have positive values of both $X$ and $B$, hence we remove the negative sign.

In [126]:
datadir = joinpath("ieee_test_cases") 
gens = CSV.read(joinpath(datadir,"Gen14.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran14_b.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load14.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters
lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines2.f
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

lines

,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.01938,0.05917,1,10000,1,16.9005
2,1,5,0.05403,0.22304,1,10000,2,4.4835
3,2,3,0.04699,0.19797,1,10000,3,5.05127
4,2,4,0.05811,0.17632,1,10000,4,5.67151
5,2,5,0.05695,0.17388,1,10000,5,5.75109
6,3,4,0.06701,0.17103,1,10000,6,5.84693
7,4,5,0.01335,0.04211,1,10000,7,23.7473
8,4,7,0.0,0.20912,1,10000,8,4.78194
9,4,9,0.0,0.55618,1,10000,9,1.79798


In [84]:
lines.b = lines.reactance ./ (lines.reactance.^2 + lines.resistance.^2);
lines.g = lines.resistance ./ (lines.reactance.^2 + lines.resistance.^2);

In [85]:
lines

,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b,g
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,1,2,0.01938,0.05917,1,10000,1,15.2631,4.99913
2,1,5,0.05403,0.22304,1,10000,2,4.23498,1.0259
3,2,3,0.04699,0.19797,1,10000,3,4.78186,1.13502
4,2,4,0.05811,0.17632,1,10000,4,5.11584,1.68603
5,2,5,0.05695,0.17388,1,10000,5,5.19393,1.70114
6,3,4,0.06701,0.17103,1,10000,6,5.06882,1.98598
7,4,5,0.01335,0.04211,1,10000,7,21.5786,6.84098
8,4,7,0.0,0.20912,1,10000,8,4.78194,0.0
9,4,9,0.0,0.55618,1,10000,9,1.79798,0.0


In [86]:
gens

,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64
1,1,0.0430293,20,0,300,0,100,-100,161.62,1
2,2,0.25,35,0,140,0,100,-100,97.47,2


In [102]:
loads

,connnode,demand
,Int64,Float64
1,2,-32.55
2,3,-141.3
3,4,-71.7
4,5,-11.4
5,6,-16.8
6,9,-44.25
7,10,-13.5
8,11,-5.25
9,12,-9.15


The structure of these data are different than the above case formats, hence we write a modified solver function:

In [88]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode)))
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[N]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])

    # Flow constraints on each branch; 
    # In DCOPF, line flow is a function of voltage angles
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineFlows = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines)) 
    for l in 1:nrow(lines)
        cLineFlows[l] = @constraint(DCOPF, 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
        )
    end
    
    # Max line flow limits
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineLimits = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines)) 
    for l in 1:nrow(lines)
        cLineLimits[l] = @constraint(DCOPF,
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
        ) 
    end

    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [89]:
solution = dcopf_ieee(gens, lines, loads);

Presolving model
46 rows, 47 cols, 140 nonzeros
41 rows, 42 cols, 130 nonzeros
Presolve : Reductions: rows 41(-55); columns 42(-182); elements 130(-74)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.0329890042e+00 Pr: 41(743849); Du: 0(7.01557e-12) 0s
         42     5.1800000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 42
Objective value     :  5.1800000000e+03
HiGHS run time      :          0.00


In [90]:
solution.generation

,node,gen
,Int64,Float64
1,1,259.0
2,2,0.0


In [91]:
solution.angles

14-element Vector{Float64}:
  0.0
 -0.11768346358632928
 -0.26490396022749335
 -0.21794788882459049
 -0.1874356957372686
 -0.29370399556841936
 -0.2784511797657695
 -0.2784511797657695
 -0.3102796372850843
 -0.31612353087789996
 -0.30944033405589544
 -0.3166582896510005
 -0.32194343394642977
 -0.34321391954752184

In [92]:
solution.prices

,node,value
,Int64,Float64
1,1,20.0
2,2,20.0
3,3,20.0
4,4,20.0
5,5,20.0
6,6,20.0
7,7,20.0
8,8,20.0
9,9,20.0


Now let's reduce capacity along one of the lines to cause some network congestions...

In [130]:
# set capacity of line 1 to 100, which prevents Generator 1 from serving all loads
lines[1,:capacity] = 100; 
solution = dcopf_ieee(gens, lines, loads);

Presolving model
46 rows, 47 cols, 140 nonzeros
39 rows, 40 cols, 129 nonzeros
Presolve : Reductions: rows 39(-57); columns 40(-184); elements 129(-75)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.7940680706e+00 Pr: 39(534743); Du: 0(9.27713e-12) 0s
         41     6.6370239604e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 41
Objective value     :  6.6370239604e+03
HiGHS run time      :          0.00


In [131]:
solution.generation

,node,gen
,Int64,Float64
1,1,161.865
2,2,97.1349


In [132]:
solution.prices

,node,value
,Int64,Float64
1,1,20.0
2,2,35.0
3,3,33.3625
4,4,31.9478
5,5,30.9285
6,6,31.2751
7,7,31.765
8,8,31.765
9,9,31.6689
